In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.animation as animation
from matplotlib import rc
from parcels import plotTrajectoriesFile
from parcels.plotting import create_parcelsfig_axis, cartopy_colorbar
import cartopy
%pylab inline

In [ ]:
class ParticleData(object):
    def __init__(self):
        self.id = []

def load_particles_file(fname, varnames):
    T = ParticleData()
    pfile = xr.open_dataset(str(fname), decode_cf=True)
    T.id = pfile.variables['trajectory'][:]

    for v in varnames:
        setattr(T, v,  np.ma.filled(pfile.variables[v][:], np.nan))
    pfile.close()
    
    return T

D = {}
wstokes = False
for y in [2001]:
    if wstokes:
        fname = "medturtles_%d_wstokes.nc" % y
    else:
        fname = "medturtles_%d.nc" % y

    D[y] = load_particles_file(fname, ['lon', 'lat', 'time'])
    D[y].year = y
    D[y].fname = fname

In [ ]:
from matplotlib import animation, rc
pylab.rcParams['figure.figsize'] = (11, 8)

for y in D:
    T = D[y]
    projection = cartopy.crs.PlateCarree()
    fig, ax = plt.subplots(1, 1,subplot_kw={'projection': projection})
    ax.coastlines(resolution='50m')
    ax.add_feature(cartopy.feature.LAND)

    plottimes = np.arange(np.min(T.time), np.max(T.time), dtype='datetime64[1D]')

    b = np.where(np.logical_and(T.time >= plottimes[0], T.time < plottimes[1]))
    scat = ax.scatter(T.lon[b], T.lat[b], s=3, 
                      transform=cartopy.crs.Geodetic(), cmap="bwr")
    axextent = [10, 38, 30, 43]
    ax.set_extent(axextent, crs=cartopy.crs.PlateCarree())

    ttl = ax.set_title(T.fname+' particles on ' + str(plottimes[0]))
    frames = np.arange(0, len(plottimes)-1)

    def animate(t):
        b = np.where(np.logical_and(T.time >= plottimes[t], T.time < plottimes[t+1]))
        scat.set_offsets(np.vstack((T.lon[b], T.lat[b])).transpose())
        ttl.set_text(T.fname+' particles on ' + str(plottimes[t]))
        return scat,

    ax.set_extent(axextent, crs=cartopy.crs.PlateCarree())
    anim = animation.FuncAnimation(fig, animate, frames=frames, interval=100, blit=True)
    anim
    mname = fname.replace('.nc', '.gif')
    anim.save(mname, writer='imagemagick', fps=10)